In [ ]:
%%capture
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import timeit

import delfi.distribution as dd
import delfi.inference as infer
import delfi.generator as dg

from delfi.simulator import GaussMixture
import delfi.summarystats as ds
from delfi.utils.viz import plot_pdf

# problem setup

In [ ]:
#p_true = dd.MoG(a=[0.5, 0.5], ms=[np.asarray([5., 5.]), np.asarray([-5., -5.])], Ss=[1.0*np.eye(2), 1.0*np.eye(2)])
#p_true.ndim=2

seed=42

return_abs = False

# basic approach to controlling generator seeds
def init_g(seed):
    m = GaussMixture(dim=2, bimodal=True, return_abs=return_abs, noise_cov=[0.01, 0.01], seed=seed)
    p = dd.Uniform(lower=[-1,-1], upper=[1,1], seed=seed)
    s = ds.Identity()
    return dg.Default(model=m, prior=p, summary=s)

g = init_g(seed=seed)

obs_stats = np.array([[.5, -.5]])

trn_data = g.gen(1000)
plt.subplot(1,2,1)
plt.plot(trn_data[1], trn_data[0], '.')
plt.show()

# SNPE parameters
    
n_components = 2
    
# training schedule
n_train=1000
n_rounds=5

# fitting setup
minibatch=100
epochs=300

# network setup
n_hiddens=[20,20]
reg_lambda=0.01

# convenience
pilot_samples=0
svi=False
verbose=True
prior_norm=False
init_norm=False


# discrete-proposal SNPE-C (MAF)

In [ ]:
monitor_every = min(10 ** 5 // float(n_train), 1.0) # output
n_mades = 5 # number of MADES
act_fun = 'tanh'
rng = np.random
rng.seed(seed)
batch_norm = False
n_null = 1

g = init_g(seed=seed)

res = infer.SNPEC(g, 
                 obs=obs_stats, 
                 n_hiddens=n_hiddens, 
                 seed=seed, 
                 reg_lambda=reg_lambda,
                 pilot_samples=pilot_samples,
                 svi=svi,
                 n_mades=n_mades,
                 act_fun=act_fun,
                 rng=rng, # this needs to be taken from infernece object, not given extra !
                 batch_norm=batch_norm, 
                 verbose=verbose,
                 prior_norm=prior_norm)

t = timeit.time.time()

logs, tds, posteriors = res.run(n_train=n_train, 
                    proposal='discrete',
                    moo='p_tilda',
                    n_null = n_null,
                    n_rounds=n_rounds, 
                    minibatch=minibatch, 
                    epochs=epochs)

print(timeit.time.time() -  t)


In [ ]:
from delfi.utils.viz import probs2contours

model = res.network

for r in range(n_rounds):
    plt.plot(logs[r]['loss'])
    plt.show()

xo = 1.*obs_stats.flatten()
lims = np.array([[-1,1], [-1,1]])
i,j = 0,1
resolution = 100

p_true = dd.MoG(a=[0.5, 0.5], ms=[np.asarray([.5, -.5]), np.asarray([-.5, .5])], Ss=[0.01*np.eye(2), 0.01*np.eye(2)])
p_true.ndim=2

xx = np.linspace(lims[i, 0], lims[i, 1], resolution)
yy = np.linspace(lims[j, 0], lims[j, 1], resolution)
X, Y = np.meshgrid(xx, yy)
xy = np.concatenate(
    [X.reshape([-1, 1]), Y.reshape([-1, 1])], axis=1)
pp = model.eval((xo, xy), log=False).reshape(list(X.shape))

plt.imshow(pp.T, origin='lower',
           extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
           aspect='auto', interpolation='none')
pp = p_true.eval(xy, log=False).reshape(list(X.shape))
plt.contour(Y, X, probs2contours(pp, levels=(0.68, 0.95)), levels=(0.68, 0.95), colors=('w', 'y'))
plt.show()

## single round, 'manual' fit

In [ ]:
"""
from delfi.inference.SNPEC import snpec_maf_loss_discrete_proposal_null
from delfi.neuralnet.Trainer import Trainer, ActiveTrainer
import snl
from snl.ml.models.mafs import ConditionalMaskedAutoregressiveFlow

n_inputs  = 2 # dimensionality of summary stats
n_outputs = 2 # dimensionality of parameters
monitor_every = min(10 ** 5 // float(n_train), 1.0) # output
n_mades = 5 # number of MADES
act_fun = 'tanh'
rng = np.random
rng.seed(seed)

n_null = 1

model = ConditionalMaskedAutoregressiveFlow(
                n_inputs=n_inputs,
                n_outputs=n_outputs,
                n_hiddens=n_hiddens,
                act_fun=act_fun,
                n_mades=n_mades,
                mode='random',
                batch_norm=False,
                rng=rng)

model.aps = model.parms

loss, trn_inputs = snpec_maf_loss_discrete_proposal_null(maf=model, svi=False)

g = init_g(seed=seed)
trn_data = g.gen(n_train)

lprs = np.log(1.)-g.prior.eval(trn_data[0],log=True)
trn_data = (trn_data[0], trn_data[1], lprs)

t_ = timeit.time.time()

t = ActiveTrainer(model, 
                  loss,
                  trn_data=trn_data, 
                  trn_inputs=trn_inputs,
                  seed=None,
                  monitor=None)
print('compilation time', timeit.time.time() -  t_)

t_ = timeit.time.time()
log = t.train(
    proposal=g.prior,
    prior=g.prior,
    moo='p_tilda',
    obs=None,
    n_null=n_null,
    epochs=500, 
    minibatch=100, 
    stop_on_nan=True,
    verbose=False)

print(timeit.time.time() -  t_)
"""

# SNL (MAFs)

In [ ]:
import sys
import snl.inference.nde as nde
import snl.ml.models.mafs as mafs
from delfi.utils.delfi2snl import SNLprior, SNLmodel

n_inputs  = 2 # dimensionality of summary stats
n_outputs = 2 # dimensionality of parameters
monitor_every = min(10 ** 5 // float(n_train), 1.0) # output
n_mades = 5 # number of MADES
act_fun = 'tanh'
rng = np.random
rng.seed(seed)

model = mafs.ConditionalMaskedAutoregressiveFlow(
                n_inputs=n_inputs,
                n_outputs=n_outputs,
                n_hiddens=n_hiddens,
                act_fun=act_fun,
                n_mades=n_mades,
                mode='random',
                rng=rng)

# generator
g = init_g(seed=seed)

# inference object
inf = nde.SequentialNeuralLikelihood(SNLprior(g.prior),               # method to draw parameters  
                                     SNLmodel(g.model, g.summary).gen # method to draw summary stats
                                    )

# train
t = timeit.time.time()

rng = np.random
rng.seed(seed)
model = inf.learn_likelihood(obs_stats.flatten(), model, n_samples=n_train, n_rounds=n_rounds, 
                             train_on_all=True, thin=10, save_models=False, 
                             logger=sys.stdout, rng=rng)

print(timeit.time.time() -  t)


### visualize learned likelihood

In [ ]:
import snl.inference.mcmc as mcmc
log_posterior = lambda t: model.eval([t, obs_stats.flatten()]) + inf.prior.eval(t)
sampler = mcmc.SliceSampler(x=inf.all_ps[-1][-1], lp_f=log_posterior, thin=10)

ps = sampler.gen(1000)

In [ ]:
xo = 1.*obs_stats.flatten()

lims = np.array([[-1,1], [-1,1]])
i,j = 0,1
resolution = 50
        
xx = np.linspace(lims[i, 0], lims[i, 1], resolution)
yy = np.linspace(lims[j, 0], lims[j, 1], resolution)
X, Y = np.meshgrid(xx, yy)
xy = np.concatenate(
    [X.reshape([-1, 1]), Y.reshape([-1, 1])], axis=1)
pp = model.eval((xo, xy), log=False).reshape(list(X.shape))

plt.imshow(pp.T, origin='lower',
           extent=[lims[j, 0], lims[j, 1], lims[i, 0], lims[i, 1]],
           aspect='auto', interpolation='none')
#plt.savefig('/home/mackelab/Desktop/SNL_5rounds_final_likelihood_tightll.pdf')
plt.show()

In [ ]:
plt.figure(figsize=(16,11))
for r in range(n_rounds):
    plt.subplot(2,3,(r+1))
    plt.plot(inf.all_ps[r][:,0],
             inf.all_ps[r][:,1], 'k.')
    plt.axis([-1,1,-1,1])
    plt.xlabel('theta1')
    plt.xlabel('theta2')
    plt.title('round r='+str(r))

plt.subplot(2,3,6)
plt.plot(ps[:,0],
         ps[:,1], 'k.')
plt.axis([-1,1,-1,1])
plt.xlabel('theta1')
plt.xlabel('theta2')
plt.title('round r='+str(n_rounds))
#plt.savefig('/home/mackelab/Desktop/posteriors_per_round_tightll.pdf')
    
plt.show()

# SNPE A (always MDNs)

In [ ]:
g = init_g(seed=seed)

res_A = infer.CDELFI(g, 
                 obs=obs_stats, 
                 n_hiddens=n_hiddens, 
                 n_components=n_components,
                 seed=seed, 
                 reg_lambda=reg_lambda,
                 pilot_samples=pilot_samples,
                 svi=svi,
                 verbose=verbose,
                 init_norm=init_norm,
                 prior_norm=prior_norm)

t = timeit.time.time()

logs_A, tds_A, posteriors_A = res_A.run(n_train=n_train, 
                    n_rounds=n_rounds, 
                    minibatch=minibatch, 
                    epochs=epochs)

print(timeit.time.time() -  t)


# discrete-proposal SNPE C (MDNs)

In [ ]:
g = init_g(seed=seed)

res_C = infer.SNPEC(g, 
                 obs=obs_stats, 
                 n_hiddens=n_hiddens, 
                 n_components=n_components,
                 seed=seed, 
                 reg_lambda=reg_lambda,
                 pilot_samples=pilot_samples,
                 svi=svi,
                 verbose=verbose,
                 prior_norm=prior_norm)

t = timeit.time.time()

logs_C, tds_C, posteriors_C = res_C.run(n_train=n_train, 
                    proposal='discrete',
                    moo='resample',
                    n_null = n_null,
                    n_rounds=n_rounds, 
                    minibatch=minibatch, 
                    epochs=epochs)

print(timeit.time.time() -  t)


## direct comparison SNPE-C against SNPE-A (both MDN)

In [ ]:
for r in range(n_rounds):
    fig,_ = plot_pdf(posteriors_A[r],
             pdf2=posteriors_C[r],
             lims=[[-1,1],[-1,1]],
             ticks=True,             
             #pdf2=g.prior,
             #gt=np.array([5,-5]), 
             resolution=100,
             figsize=(16,16));
    fig.suptitle('final posterior estimate vs MCMC samples and prior', fontsize=14)
    fig.show()
